In [1]:
from s3fs import S3FileSystem
from os import getenv

In [2]:
import dotenv

dotenv.load_dotenv()

True

In [3]:
s3 = S3FileSystem(
    key=getenv("S3_ACCESS_KEY_ID"),
    secret=getenv("S3_SECRET_ACCESS_KEY"),
    endpoint_url=getenv("S3_ENDPOINT_URL"),
)


In [4]:
import pandas as pd

In [37]:
path = "s3a://data-ice-lake-05/dev/source/transaction-service/object_type=transaction/date=2023-10-07/hour=18/batch_id=0"

s3.ls(path)

['data-ice-lake-05/dev/source/transaction-service/object_type=transaction/date=2023-10-07/hour=18/batch_id=0/part-00000-4c34a093-a26c-4c5d-bdc3-214ed33f7058.c000.snappy.parquet']

In [38]:
with s3.open(s3.ls(path)[0], 'rb') as f:
    df = pd.read_parquet(f)

In [39]:
df.head()

,object_id,sent_dttm,payload,trigger_dttm
0,e2d4cfd9-4dc6-4a02-8c7a-c391a420206b,2022-10-01 00:11:45,"{""operation_id"":""e2d4cfd9-4dc6-4a02-8c7a-c391a...",2023-10-07 18:44:34.385775
1,b05f56ef-3b59-49ee-851c-a37a98d1dd28,2022-10-01 00:22:28,"{""operation_id"":""b05f56ef-3b59-49ee-851c-a37a9...",2023-10-07 18:44:34.385775
2,a62136ce-f65a-45d8-8472-18bdc5c1e0e3,2022-10-01 00:28:04,"{""operation_id"":""a62136ce-f65a-45d8-8472-18bdc...",2023-10-07 18:44:34.385775
3,a0d7ccfa-8bf1-4921-8d68-a50e5e2b2fbd,2022-10-01 00:29:53,"{""operation_id"":""a0d7ccfa-8bf1-4921-8d68-a50e5...",2023-10-07 18:44:34.385775
4,818b7011-e7e0-48c1-a0e3-8c061c92fc1f,2022-10-01 00:38:23,"{""operation_id"":""818b7011-e7e0-48c1-a0e3-8c061...",2023-10-07 18:44:34.385775


In [9]:
path = "s3a://data-ice-lake-05/dev/clean/transaction-service/сurrencies/date=2023-10-08/hour=7"

s3.ls(path)

['data-ice-lake-05/dev/clean/transaction-service/сurrencies/date=2023-10-08/hour=7/part-00000-ff59e34a-c639-4c0f-8eb1-69b47c880b64.c000.snappy.parquet']

In [10]:
with s3.open(s3.ls(path)[0], 'rb') as f:
    df = pd.read_parquet(f)

In [11]:
df.dtypes

object_id                     object
date_update           datetime64[ns]
currency_code                  int32
currency_code_with             int32
currency_with_div            float32
trigger_dttm          datetime64[ns]
dtype: object

In [12]:
df.head()

,object_id,date_update,currency_code,currency_code_with,currency_with_div,trigger_dttm
0,7e302cf9-704c-5b89-9b9d-afc2d44f5a48,2022-10-22,460,430,0.96,2023-10-08 07:30:29.714301
1,cb0a3884-f516-5a18-8533-fd36f88cd870,2022-10-28,440,420,0.91,2023-10-08 07:30:29.714301
2,553e011a-c1e1-5b30-882a-72853d95812c,2022-10-31,410,470,1.09,2023-10-08 07:30:29.714301
3,03ce4653-572e-524c-8c70-44e38b0ac550,2022-10-10,430,460,0.96,2023-10-08 07:30:29.714301
4,9eb2613e-4f9a-5e29-9499-62305d571604,2022-10-20,430,420,1.05,2023-10-08 07:30:29.714301


In [40]:
df['payload'].iloc[0]

'{"operation_id":"e2d4cfd9-4dc6-4a02-8c7a-c391a420206b","account_number_from":914810,"account_number_to":876839,"currency_code":430,"country":"russia","status":"done","transaction_type":"sbp_incoming","amount":290000,"transaction_dt":"2022-10-01 00:11:45"}'

In [15]:
df['amount'].unique()

array([ 22600, 250000,   5000, ...,  30400, 215400, 679600])

In [21]:
print(f"{df['amount'].min():_}")

-3_849_206


In [22]:
print(f"{df['amount'].max():_}")

5_930_700


In [23]:
print(f"{df['amount'].mean():_}")

179_935.33373337335


In [24]:
print(f"{df['amount'].median():_}")

100_000.0


In [13]:
df['amount'] = df['amount'].astype('int64')

In [14]:
df.dtypes

operation_id                   object
account_number_from           float64
account_number_to             float64
currency_code                   int32
country                        object
status                         object
transaction_type               object
amount                          int64
transaction_dt         datetime64[ns]
trigger_dttm           datetime64[ns]
dtype: object

In [10]:
df.date_update.unique()

<DatetimeArray>
['NaT']
Length: 1, dtype: datetime64[ns]

In [11]:
df.shape

(1680, 6)

In [14]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [16]:
df.sort_values('object_id').head(20)

,object_id,date_update,currency_code,currency_code_with,currency_with_div,trigger_dttm
1307,00519ce2-066d-5d42-a5dd-ccb85f100f31,NaT,460,470,1.06,2023-10-03 05:30:47.659694
125,008724ab-dcef-5d9f-99b9-d04023aff9ae,NaT,420,430,0.96,2023-10-03 05:30:47.659694
199,00b0ebd9-41c2-5f62-a4c3-c3acb9a25afa,NaT,440,460,1.02,2023-10-03 05:30:47.659694
1585,0106e23b-f680-514e-b712-dfb7cd5282b5,NaT,410,460,0.97,2023-10-03 05:30:47.659694
1338,010e8d85-56d7-5e74-a28e-4fdf226ea695,NaT,450,440,1.01,2023-10-03 05:30:47.659694
949,012116b6-61f1-5b14-8420-6aa6811af7a8,NaT,410,450,1.07,2023-10-03 05:30:47.659694
1090,01276f13-b24f-58c3-b096-2ba50de97159,NaT,470,430,1.03,2023-10-03 05:30:47.659694
94,0198e79b-afb9-5907-9ebf-559bb166fe31,NaT,440,430,1.09,2023-10-03 05:30:47.659694
813,01c4eb2d-0076-50f9-a952-dfeda04d8b48,NaT,430,420,0.96,2023-10-03 05:30:47.659694
10,0207a732-4c37-563e-92f7-7a8669ab2861,NaT,450,440,1.05,2023-10-03 05:30:47.659694


In [17]:
df['row_number'] = df.groupby('object_id').cumcount() + 1

In [19]:
df[df['row_number'] > 1].head()

,object_id,date_update,currency_code,currency_code_with,currency_with_div,trigger_dttm,row_number
